In [12]:
# Importing libraries
from bs4 import BeautifulSoup
import requests
import psycopg2
import becas

# Starting a connection with postgres
conn = psycopg2.connect("dbname=postgres user=postgres")
conn.autocommit = True
# Making a cursor object
cur = conn.cursor()
#cur.execute("DROP TABLE GSE_data")
# Create table in database
cur.execute("CREATE TABLE GSE_data(GSE_Id varchar(7) PRIMARY KEY, Title text, Summary text, Organism text, Experiment_type text, Overall_design text, Citation text, Keyword text [], Unique_Identifier text [])")


# Required Authentication for becas API
becas.email = "kanshul03@gmail.com"


# Create a list of all the GSE_ID's whose information to scrape
GSE_Id = ["63312", "78224", "74018", "50734", "114644", "60477", "53599", "80582", "109493", "35200"]
for id in GSE_Id:
    # Request resources from the website
    response = requests.get("https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE{}".format(id))
    content = response.content
    # Parse the content from the website using BeautifulSoup
    parser = BeautifulSoup(content, "html.parser")
    # Below line will find all the occurances of <td> using the parser created above
    td = parser.find_all("td")
    
    # Create a dictionary to store all the information regarding a GSE_ID
    dic = {"GSE Id": id}
    # Parsing over the required content in the website and saving it in dictionary
    for i in range(38,99,2):
        dic[td[i].text] = td[i+1].text
    
    
    # List of elements that are required related to GSE_ID
    info = ["GSE Id", "Title", "Summary", "Organism", "Experiment type", "Overall design", "Citation(s)"]
    # Make a seperate dictionary with only the required elements as key and their description as values
    req_dic = {key: dic[key] for key in info}
    
    
    # Query to insert the collected data into database
    query = """INSERT INTO GSE_data (GSE_Id, Title, Summary, Organism, Experiment_type, Overall_design, Citation) VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    record_to_insert = (req_dic["GSE Id"], req_dic["Title"], req_dic["Summary"], req_dic["Organism"], req_dic["Experiment type"], req_dic["Overall design"], req_dic["Citation(s)"])
    cur.execute(query, record_to_insert)
    
    
    # Annotate the summary text using Becas API
    json = becas.annotate_text(req_dic["Summary"])
    # Make a list of all the entity values from the resulted dictionary(json) after annotation
    entities_list = json["entities"]
    
    # Make empty list for keywords and unique_identifiers
    keyword = []
    unique_identifier = []
    
    for each in entities_list:
        # Split every element of entity_list into three parts using split on "|"
        each = each.split("|")
        # The first part is keyword
        keyword.append(each[0])
        # The second part is Unique Identifier
        # Many Keywords have multiple identifiers seperated by ";"
        # So split them and append each to the list
        each[1] = each[1].split(";")
        for identifier in each[1]:
            unique_identifier.append(identifier)
    
    
    # Query to add keywords and unique identifiers in the table for each GSE ID
    query = ("""UPDATE GSE_Data 
                SET Keyword = %s,
                unique_identifier = %s
                WHERE GSE_Id = %s
                """)
    record_to_insert = (keyword, unique_identifier, id)
    cur.execute(query, record_to_insert)

    
# Query to find GSE_Id's having Disease keyword.    
cur.execute("SELECT GSE_Id FROM GSE_Data WHERE array_to_string(unique_identifier, ',') LIKE '%DISO%'")
data = cur.fetchall()
print(data)


# Close the connection with the database
conn.close()

['UMLS:C1704259:T044:PROC', 'UMLS:C0007621:T191:DISO', 'UMLS:C0023467:T191:DISO', 'UNIPROT:P41212:T116:PRGE', 'UMLS:C0023467:T191:DISO', 'UMLS:C0023467:T191:DISO', 'UMLS:C0023418:T191:DISO', 'NCBI:9606:T001:SPEC', 'UNIPROT:Q03164:T116:PRGE', 'UNIPROT:O60861:T116:PRGE', 'UNIPROT:P01111:T116:PRGE', 'GO:0010467::PROC', 'CHEBI:62488:T103:CHED', 'UMLS:C0023418:T191:DISO', 'GO:0010467::PROC', 'UNIPROT:P01116:T116:PRGE', 'CHEBI:63620:T103:CHED', 'UNIPROT:P01112:T116:PRGE', 'UNIPROT:P01116:T116:PRGE', 'CHEBI:63620:T103:CHED', 'UNIPROT:P01112:T116:PRGE', 'UMLS:C1956422:T025:ANAT', 'CHEBI:63620:T103:CHED', 'UNIPROT:Q03164:T116:PRGE', 'UNIPROT:O60861:T116:PRGE', 'UMLS:C1378511:T191:DISO', 'UMLS:C0280141:T191:DISO', 'UMLS:C1704259:T044:PROC', 'UMLS:C0023467:T191:DISO', 'GO:0007165::PROC', 'GO:0023052::PROC', 'CHEBI:35222:T103:CHED', 'UNIPROT:P45985:T116:PRGE', 'UNIPROT:O14733:T116:PRGE', 'CHEBI:28398:T103:CHED', 'UNIPROT:Q02750:T116:PRGE', 'UNIPROT:P52564:T116:PRGE', 'EC:2.7.12.2:T126:ENZY', 'UNIP